In [2]:
''' 
A Amazônia, atualmente responsável por um terço das florestas tropicais do mundo (IPAM), abriga mais de 
10% de todas as espécies conhecidas (Greenpeace). Este bioma é fundamental para a regulação dos corpos d'água, os padrões 
climáticos, as culturas indígenas e ribeirinhas, entre muitos outros aspectos vitais. No entanto, ele está 
sob constante ameaça devido às ações humanas que afetam negativamente o meio ambiente. Além da Amazônia, 
outros biomas que compõem o território brasileiro também enfrentam desafios significativos.

Diante das queimadas que assolam o Brasil, propõe-se uma comparação entre a intensidade das queimadas nos estados que 
abrangem a Amazônia e nos demais estados brasileiros nas últimas décadas.

Para essa análise, utiliza-se a Cobertura Média de Dossel Queimado, uma medida ponderada que reflete a severidade das 
queimadas com base nos diferentes níveis de cobertura vegetal. O cálculo da cobertura média é dado pela fórmula:

Cobertura Média de Dossel Queimado = (∑(Área Queimada × Percentual de Cobertura de Dossel))/Área Total Queiamda

Esse indicador permite a comparação entre estados, oferecendo uma visão clara da intensidade das queimadas em 
termos de degradação da cobertura florestal. Ao utilizá-lo, é possível avaliar a severidade das queimadas em diferentes 
regiões e compreender melhor o impacto ambiental nos biomas brasileiros, especialmente na Amazônia

'''
#tratamento da nossa base de dados considerando a Cobertura Média de Dossel Queimado como 'COB'
import pandas as pd
df = pd.read_excel('BRAdados.xlsx', sheet_name = 'COB', index_col='Ano')
df_melt =pd.melt(df, value_vars = df.columns, var_name = 'Estado', value_name = 'COB', ignore_index=False)
df_melt = df_melt.reset_index()
df_melt.head()


,Ano,Estado,COB
0,2001,AC,0.275280
1,2002,AC,0.279130
2,2003,AC,0.277913
3,2004,AC,0.278809
4,2005,AC,0.276839


In [3]:
#importação de bibliotecas relevantes
import plotly as plt
import plotly.express as px
import json
from urllib.request import urlopen

In [4]:
#carregue os dados geoespaciais dos estados do Brasil com o GeoJSON
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
 Brazil = json.load(response);

In [5]:
#criar uma dicionário que mapeie as siglas dos estados brasileiros e seus nomes
state_id_map = {}
for feature in Brazil ['features']:
 feature['id'] = feature['properties']['name']
 state_id_map[feature['properties']['sigla']] = feature['id']
print(state_id_map)

{'AC': 'Acre', 'AL': 'Alagoas', 'AM': 'Amazonas', 'AP': 'Amapá', 'BA': 'Bahia', 'CE': 'Ceará', 'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão', 'MG': 'Minas Gerais', 'MS': 'Mato Grosso do Sul', 'MT': 'Mato Grosso', 'PA': 'Pará', 'PB': 'Paraíba', 'PE': 'Pernambuco', 'PI': 'Piauí', 'PR': 'Paraná', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RO': 'Rondônia', 'RR': 'Roraima', 'RS': 'Rio Grande do Sul', 'SC': 'Santa Catarina', 'SE': 'Sergipe', 'SP': 'São Paulo', 'TO': 'Tocantins', 'DF': 'Distrito Federal'}


In [6]:
#indique os valores mínimo e máximo que a variável COB assume para serem usados a posteriori como parâmetros
min_value = df_melt['COB'].min()
max_value = df_melt['COB'].max()

#criar o mapa coroplético com valores para variável de interesse (COB), com controle deslizante para os anos e propriedade interativa com o mouse
fig = px.choropleth(df_melt,
                   locations='Estado',
                   geojson=Brazil,
                    featureidkey='properties.sigla',
                   color='COB',
                   hover_name='Estado',
                   title='Cobertura Média Ponderada das Áreas Queimadas',
                   animation_frame = 'Ano',
                   color_continuous_scale=px.colors.sequential.amp,
                   range_color=[min_value, max_value])
fig.update_geos(fitbounds='locations', visible=False)
fig;

In [7]:
#gerar link para arquivo html com a imagem anterior para baixá-lo
from IPython.display import FileLink
fig.write_html("mapa_cobertura_queimada.html")
display(FileLink('mapa_cobertura_queimada.html'))

C:\Users\Dell\mapa_cobertura_queimada.html

In [8]:
#tratamento da base de dados para análise de CO2 emitido pelas queimadas
df_2 = pd.read_excel('BRAdados.xlsx', sheet_name='CO2')
df_2 = df_2.drop(['umd_tree_cover_density_2000__threshold'], axis=1)
df_2 = df_2.fillna(0)
df_2 = df_2.groupby('subnational1').sum().reset_index()
df_2 = df_2.rename(columns={'subnational1': 'Estado'})
df_2 = pd.melt(df_2, id_vars=['Estado'], var_name='Ano', value_name='CO2')
df_2 = df_2.set_index('Ano') 
df_2 = df_2.reset_index()
df_2

,Ano,Estado,CO2
0,2001,Acre,128889667.0
1,2001,Alagoas,4387575.0
2,2001,Amapá,27652609.0
3,2001,Amazonas,152954351.0
4,2001,Bahia,92751667.0
...,...,...,...
616,2023,Roraima,98931642.0
617,2023,Santa Catarina,118979666.0
618,2023,Sergipe,1817577.0
619,2023,São Paulo,119754931.0


In [9]:
#indique os valores mínimo e máximo que a variável CO2 assume para serem usados a posteriori como parâmetros
min_value2 = df_2['CO2'].min()
max_value2 = df_2['CO2'].max()

#criar o mapa coroplético com valores para variável de interesse (CO2), com controle deslizante para os anos e propriedade interativa com o mouse
fig2 = px.choropleth(df_2,
                   locations='Estado',
                   geojson=Brazil,
                    featureidkey='properties.name',
                   color='CO2',
                   hover_name='Estado',
                   title='Emissões de Dióxido de Carbono causadas por perda de florestas por Estado',
                   animation_frame = 'Ano',
                   color_continuous_scale=px.colors.sequential.YlGnBu,
                    range_color=[min_value2, max_value2])
fig2.update_geos(fitbounds='locations', visible=False)
fig2;

In [10]:
#gerar link para arquivo html com a imagem anterior para baixá-lo
from IPython.display import FileLink
fig2.write_html("mapa_emissoes_CO2.html")
display(FileLink("mapa_emissoes_CO2.html"))

C:\Users\Dell\mapa_emissoes_CO2.html

In [11]:
#criação de dataframe com os Estados e seus respectivos tamannhos
dados_estados = {
    'Estado': [
        'Amazonas', 'Pará', 'Mato Grosso', 'Minas Gerais', 'Bahia', 'Mato Grosso do Sul', 
        'Goiás', 'Maranhão', 'Rio Grande do Sul', 'Tocantins', 'Piauí', 'São Paulo', 
        'Rondônia', 'Roraima', 'Paraná', 'Acre', 'Ceará', 'Amapá', 'Pernambuco', 
        'Santa Catarina', 'Paraíba', 'Rio Grande do Norte', 'Espírito Santo', 
        'Rio de Janeiro', 'Alagoas', 'Sergipe', 'Distrito Federal'
    ],
    'Área (km²)': [
        1570745.7, 1247689.5, 903366.2, 586528.3, 564733.2, 357125.0, 340111.4, 
        331983.3, 281748.5, 277720.5, 251577.7, 248219.6, 237590.7, 224300.5, 
        199307.9, 164123.7, 148920.5, 142814.6, 98067.9, 95730.7, 56467.2, 
        52811.0, 46095.6, 43796.5, 27848.0, 21915.1, 5779.9
    ]
}

df_estados_area = pd.DataFrame(dados_estados)
df_estados_area

,Estado,Área (km²)
0,Amazonas,1570745.7
1,Pará,1247689.5
2,Mato Grosso,903366.2
3,Minas Gerais,586528.3
4,Bahia,564733.2
5,Mato Grosso do Sul,357125.0
6,Goiás,340111.4
7,Maranhão,331983.3
8,Rio Grande do Sul,281748.5
9,Tocantins,277720.5


In [12]:
#mesclar dataframe original com dataframe criado no passo anterior
df_3 = df_2.copy()
df_3 = df_3.merge(df_estados_area, on='Estado', how='left')
df_3['CO2 por km²'] = df_3['CO2'].div(df_3['Área (km²)'], axis=0)
df_3 = df_3.drop(['CO2'], axis=1)
df_3

,Ano,Estado,Área (km²),CO2 por km²
0,2001,Acre,164123.7,785.320261
1,2001,Alagoas,27848.0,157.554402
2,2001,Amapá,142814.6,193.625925
3,2001,Amazonas,1570745.7,97.376903
4,2001,Bahia,564733.2,164.239799
...,...,...,...,...
616,2023,Roraima,224300.5,441.067416
617,2023,Santa Catarina,95730.7,1242.857996
618,2023,Sergipe,21915.1,82.937199
619,2023,São Paulo,248219.6,482.455580


In [13]:
#indique os valores mínimo e máximo que a variável CO2 por km2 assume para serem usados a posteriori como parâmetros
min_value3 = df_3['CO2 por km²'].min()
max_value3 = df_3['CO2 por km²'].max()

#criar o mapa coroplético com valores para variável de interesse (CO2 por km2), com controle deslizante para os anos e propriedade interativa com o mouse
fig3 = px.choropleth(df_3,
                   locations='Estado',
                   geojson=Brazil,
                    featureidkey='properties.name',
                   color='CO2 por km²',
                   hover_name='Estado',
                   title='Emissões de Dióxido de Carbono por km² causadas por perda de florestas em cada Estado',
                   animation_frame = 'Ano',
                   color_continuous_scale=px.colors.sequential.YlGnBu,
                    range_color=[min_value3, max_value3])
fig3.update_geos(fitbounds='locations', visible=False)
fig3;

In [14]:
#gerar link para arquivo html com a imagem anterior para baixá-lo
from IPython.display import FileLink
fig3.write_html("mapa_emissoes_CO2_km2.html")
display(FileLink("mapa_emissoes_CO2_km2.html"))

C:\Users\Dell\mapa_emissoes_CO2_km2.html